In [1]:
from pyquil import Program, pyqvm, get_qc
from pyquil.gates import *

## 初期状態と測定

In [2]:
# 1qbitのqvm立ち上げ
qc = get_qc('1q-qvm')
# Programインスタンス立ち上げ時には|0>が入っている．
p = Program()
# declareで結果を格納するメモリを確保する．roはreadoutの略
# 下の例では1bitの領域を確保している．
ro = p.declare('ro', 'BIT', 1)
# MEASUREによってro[0]にqubit 0の測定結果を格納
# 今は1qbitのsystemにおいて|0>方向で測定している．systemが大きくなったときにはindexの付け方に注意．
p += MEASURE(0, ro[0])

executable = qc.compile(p)
result = qc.run(executable)
# expected to be [[0]]
print(result)

[[0]]


## Hadamard

In [3]:
p = Program()
ro = p.declare('ro', 'BIT', 1)
# Hadamardを|0>に作用させる
p += H(0)
p += MEASURE(0, ro[0])
# Hadamardを作用させたことによって測定結果は確率的になるので1000回分やってみる
p.wrap_in_numshots_loop(1000)
executable = qc.compile(p)
result = qc.run(executable)

# expected to be ~ 0.5
print(result.flatten().mean())

0.5


In [4]:
# run_and_measureとするとMEASUREからrunまでと同等
p = Program()
p += H(0)
result = qc.run_and_measure(p, trials=1000)
# ただし，qc.runではndarrayで結果が帰ってくるのに対しrun_and_measureではdictで得られる．
# このdictのkeyはqubit, valueの配列(ndarray)のi番目の要素はi番目の測定結果が格納されている．
print(result[0].mean())

0.485


## Parametricなゲート
- `qc.qam.random_seed=42`のようにrandom seedを指定すると結果が確率的になっていないように見える
    - こんな感じになる
    ```
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    ```

In [5]:
import numpy as np

qubit = 0
p = Program()
ro = p.declare("ro", "BIT", 1)
theta_ref = p.declare("theta", "REAL")
p += RX(np.pi / 2, qubit)
p += RZ(theta_ref, qubit)
p += RX(-np.pi / 2, qubit)
p += MEASURE(qubit, ro[0])

executable = qc.compile(p)

parametric_measurements = []
for theta in np.linspace(0, 2 * np.pi, 200):
    # ここでパラメータを指定できる
    bitstrings = qc.run(executable, {'theta': [theta]})
    parametric_measurements.append(bitstrings[0][0])
# listの真ん中あたりが1になる確率が高い
print(parametric_measurements)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## CNOT

In [6]:
# 1qbitのqvm立ち上げ
qc = get_qc('2q-qvm')
p = Program()
ro = p.declare("ro", "BIT", 2)
# |00> -> |10> by X
p += X(0)
# |10> -> |11> by CNOT
p += X(1).controlled(0)
p += MEASURE(0, ro[0])
p += MEASURE(1, ro[1])

executable = qc.compile(p)
result = qc.run(executable)
# expected to be [[1 1]]
print(result)

[[1 1]]


## 新しいゲートを定義

In [7]:
from pyquil.quil import DefGate
sqrt_x = np.array([[ 0.5+0.5j,  0.5-0.5j],
                   [ 0.5-0.5j,  0.5+0.5j]])

# Get the Quil definition for the new gate
# ここで，定義した行列がunitaryでない場合にはValue errorが出る．
sqrt_x_definition = DefGate("SQRT-X", sqrt_x)
# Get the gate constructor
SQRT_X = sqrt_x_definition.get_constructor()

# Then we can use the new gate
p = Program()
p += sqrt_x_definition
p += SQRT_X(0)
print(p)

DEFGATE SQRT-X:
    0.5+0.5i, 0.5-0.5i
    0.5-0.5i, 0.5+0.5i

SQRT-X 0



## Parameterizeされたゲートを定義

In [8]:
from pyquil.parameters import Parameter, quil_sin, quil_cos
# Define the new gate from a matrix
theta = Parameter('theta')
crx = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, quil_cos(theta / 2), -1j * quil_sin(theta / 2)],
    [0, 0, -1j * quil_sin(theta / 2), quil_cos(theta / 2)]
])
# Parameterizeされているので明らかにunitaryでなくても実行時までエラー吐かないことに注意
gate_definition = DefGate('CRX', crx, [theta])
CRX = gate_definition.get_constructor()

# Create our program and use the new parametric gate
p = Program()
p += gate_definition
p += H(0)
p += CRX(np.pi/2)(0, 1)
print(p)

DEFGATE CRX(%theta):
    1, 0, 0, 0
    0, 1, 0, 0
    0, 0, COS(%theta/2), -i*SIN(%theta/2)
    0, 0, -i*SIN(%theta/2), COS(%theta/2)

H 0
CRX(pi/2) 0 1



## 波動関数シミュレータ
- for debug
- 量子プロセッサでは実現できない

In [9]:
from pyquil.api import WavefunctionSimulator
wf_sim = WavefunctionSimulator()
coin_flip = Program(H(0))
wf = wf_sim.wavefunction(coin_flip)
print(wf)

(0.7071067812+0j)|0> + (0.7071067812+0j)|1>


## VQEの実装
- [original paper](https://www.nature.com/articles/ncomms5213)の実装はちょっと大変なので簡単な問題を解いてみる
- $
  H=\left(
  \begin{array}{cc}
  1 & 0 \\ 0 & -1
  \end{array}
  \right)
  $ の最小固有値$-1$を求めてみる．の最小固有値$-1$を求めてみる
    - ansatzとしては$R_y(\theta) \left|0\right>$をとってみる

In [10]:
from pyquil.gates import RY

In [11]:
from scipy.optimize import minimize

class VQE:
    def __init__(self, ex_h, qc, ansatz, optimizer, init_params, trials=1000):
        self.ex_h = ex_h
        self.qc = qc
        self.ansatz = ansatz
        self.trials = trials
        self.optimizer = optimizer
        self.init_params = init_params

    
    def expectation(self, params):
        program = self.ansatz(params)
        results = self.qc.run_and_measure(program, trials=self.trials)
        return self.ex_h(results)
                
    def minimize(self):
        return self.optimizer(self.expectation, self.init_params)

# return parameterized ansatz
def ansatz(params):
    return Program(RY(params[0], 0))

# calculate expectation value using results dict
def ex_h(results):
    s = 0
    for r in results[0]:
        if r == 0:
            s += 1
        else:
            s -= 1
    return s / len(results[0])

# minimize function f with respect to parameters p
def optimizer(f, p):
    return minimize(f, p, method='Nelder-Mead', options={"disp": True, "xatol": 1.e-2})

vqe = VQE(ex_h, get_qc("1q-qvm"), ansatz, optimizer, [np.pi/2])
print(vqe.minimize())

Optimization terminated successfully.
         Current function value: -1.000000
         Iterations: 11
         Function evaluations: 25
 final_simplex: (array([[3.14159265],
       [3.13177518]]), array([-1., -1.]))
           fun: -1.0
       message: 'Optimization terminated successfully.'
          nfev: 25
           nit: 11
        status: 0
       success: True
             x: array([3.14159265])


### Grove
実は[grove](https://github.com/rigetti/grove)を使うともっと簡単に実装可能

In [42]:
from grove.pyvqe.vqe import VQE
from pyquil import api
from pyquil.paulis import sZ

vqe = VQE(minimizer=minimize, minimizer_kwargs={'method': 'nelder-mead', 
          'options':{'xatol': 1.0e-2}})
 
init_params = [np.pi/2]
qvm = api.QVMConnection()
result = vqe.vqe_run(ansatz, sZ(0), init_params, 1000, qvm=qvm)
print(result)

                     models will be ineffective
{'x': array([3.14159265]), 'fun': -1.0}
